**How to Query the Chicago Crime Dataset (BigQuery)**

In [1]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
chicago_crime = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="chicago_crime")

In [2]:
bq_assistant = BigQueryHelper("bigquery-public-data", "chicago_crime")
bq_assistant.list_tables()

['crime']

In [3]:
bq_assistant.head("crime", num_rows=3)

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,10655248,HZ405004,2016-08-24 06:00:00+00:00,0000X W LAKE ST,0263,CRIM SEXUAL ASSAULT,AGGRAVATED: KNIFE/CUT INSTR,ALLEY,False,False,111,1,42,32,02,1175790.0,1901739.0,2016,2018-02-10 15:50:01+00:00,41.885741,-87.629916,"(41.88574102, -87.62991594)"
1,13323048,JG559307,2023-12-29 23:35:00+00:00,0000X E WACKER DR,0265,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,HOTEL / MOTEL,False,False,111,1,42,32,02,1176971.0,1902537.0,2023,2024-12-21 15:40:46+00:00,41.887904,-87.625555,"(41.887904127, -87.625554948)"
2,9797811,HX446989,2014-09-25 01:00:00+00:00,0000X E LAKE ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,111,1,42,32,02,1176981.0,1901745.0,2014,2018-02-10 15:50:01+00:00,41.885731,-87.625542,"(41.885730611, -87.625542215)"


In [4]:
bq_assistant.table_schema("crime")

,name,type,mode,description
0,unique_key,INTEGER,REQUIRED,None
1,case_number,STRING,NULLABLE,None
2,date,TIMESTAMP,NULLABLE,None
3,block,STRING,NULLABLE,None
4,iucr,STRING,NULLABLE,None
5,primary_type,STRING,NULLABLE,None
6,description,STRING,NULLABLE,None
7,location_description,STRING,NULLABLE,None
8,arrest,BOOLEAN,NULLABLE,None
9,domestic,BOOLEAN,NULLABLE,None


What categories of crime exhibited the greatest year-over-year increase between 2015 and 2016?


In [5]:
query1 = """SELECT
  primary_type,
  description,
  COUNTIF(year = 2015) AS arrests_2015,
  COUNTIF(year = 2016) AS arrests_2016,
  FORMAT('%3.2f', (COUNTIF(year = 2016) - COUNTIF(year = 2015)) / COUNTIF(year = 2015)*100) AS pct_change_2015_to_2016
FROM
  `bigquery-public-data.chicago_crime.crime`
WHERE
  arrest = TRUE
  AND year IN (2015,
    2016)
GROUP BY
  primary_type,
  description
HAVING
  COUNTIF(year = 2015) > 100
ORDER BY
  (COUNTIF(year = 2016) - COUNTIF(year = 2015)) / COUNTIF(year = 2015) DESC
        """
response1 = chicago_crime.query_to_pandas_safe(query1)
response1.head(10)

,primary_type,description,arrests_2015,arrests_2016,pct_change_2015_to_2016
0,OTHER OFFENSE,VEHICLE TITLE/REG OFFENSE,288,418,45.14
1,OTHER OFFENSE,FALSE/STOLEN/ALTERED TRP,299,418,39.80
2,HOMICIDE,FIRST DEGREE MURDER,211,287,36.02
3,NARCOTICS,FOUND SUSPECT NARCOTICS,655,846,29.16
4,ASSAULT,AGGRAVATED: HANDGUN,466,521,11.80
5,MOTOR VEHICLE THEFT,AUTOMOBILE,785,872,11.08
6,LIQUOR LAW VIOLATION,LIQUOR LICENSE VIOLATION,134,144,7.46
7,ROBBERY,ARMED: HANDGUN,232,249,7.33
8,OTHER OFFENSE,OTHER VEHICLE OFFENSE,195,204,4.62
9,MOTOR VEHICLE THEFT,THEFT/RECOVERY: AUTOMOBILE,164,171,4.27


Which month generally has the greatest number of motor vehicle thefts?


In [6]:
query2 = """SELECT
  year,
  month,
  incidents
FROM (
  SELECT
    year,
    EXTRACT(MONTH
    FROM
      date) AS month,
    COUNT(1) AS incidents,
    RANK() OVER (PARTITION BY year ORDER BY COUNT(1) DESC) AS ranking
  FROM
    `bigquery-public-data.chicago_crime.crime`
  WHERE
    primary_type = 'MOTOR VEHICLE THEFT'
    AND year <= 2016
  GROUP BY
    year,
    month )
WHERE
  ranking = 1
ORDER BY
  year DESC
        """
response2 = chicago_crime.query_to_pandas_safe(query2)
response2.head(10)

,year,month,incidents
0,2016,12,1106
1,2015,8,967
2,2014,10,922
3,2013,1,1470
4,2012,6,1469
5,2011,1,1862
6,2010,12,1880
7,2009,12,1539
8,2008,7,2015
9,2007,10,1709


In [7]:
# Random query test, was just querying for homicides and mainly using this
# to check how big this query would be an execute accordingly
queryExample = """SELECT
  date,
  primary_type,
  description,
  year
FROM
  `bigquery-public-data.chicago_crime.crime`
WHERE
  primary_type = 'HOMICIDE'
ORDER BY
  date DESC
"""

# Check the query size prior to running the query to estimate how long
# it will take to run, in this case its .358, meaning 350 MBs
# since its measured in GB's, this is easily runnable, fairly small
chicago_crime.estimate_query_size(queryExample)

0.35801383946090937

In [8]:
# This method will take the query and return a pandas dataframe
response3 = chicago_crime.query_to_pandas_safe(queryExample)
response3.head(10)

,date,primary_type,description,year
0,2025-04-09 16:40:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
1,2025-04-05 20:09:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
2,2025-03-31 12:30:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
3,2025-03-28 18:13:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
4,2025-03-27 16:51:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
5,2025-03-26 09:00:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
6,2025-03-26 05:22:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
7,2025-03-25 15:40:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
8,2025-03-24 18:42:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025
9,2025-03-24 11:36:00+00:00,HOMICIDE,FIRST DEGREE MURDER,2025


In [9]:
# You can also add another parameter as shown here which will disregard
# the call if its bigger than 250 MB, or whatever you specify
homicide_crimes = chicago_crime.query_to_pandas_safe(queryExample, 0.25)
print(homicide_crimes)

Query cancelled; estimated size of 0.35801383946090937 exceeds limit of 0.25 GB
None


In [10]:
# Convert the query response into a csv format
response3.to_csv("homicide_crimes.csv")